In [20]:
!pip install azure-ai-ml azure identity
!pip install scikit-learn pandas numpy joblib
!pip install azure-storage-blob

  Preparing metadata (setup.py) ... - error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-c5vg614q/azure_6fc8b6daa87f4babacaa218d93090f16/setup.py", line 60, in <module>
          raise RuntimeError(message)
      RuntimeError:
      
      Starting with v5.0.0, the 'azure' meta-package is deprecated and cannot be installed anymore.
      Please install the service specific packages prefixed by `azure` needed for your application.
      
      The complete list of available packages can be found at:
      https://aka.ms/azsdk/python/all
      
      Here's a non-exhaustive list of common packages:
      
      -  azure-mgmt-compute (https://pypi.python.org/pypi/azure-mgmt-compute) : Management of Virtual Machines, et

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

# Load dataset
data = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Drop customerID
data = data.drop('customerID', axis=1)

# Convert TotalCharges to numeric
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'] = data['TotalCharges'].fillna(data['TotalCharges'].median())

# Encode categorical variables
for column in data.select_dtypes(include='object').columns:
    data[column] = LabelEncoder().fit_transform(data[column])

# Split data
X = data.drop('Churn', axis=1)
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
os.makedirs("../model", exist_ok=True)
joblib.dump(model, "../model/churn_model.pkl")

print("✅ Model trained and saved successfully!")


✅ Model trained and saved successfully!


In [6]:
!pip install azure-ai-ml
!pip install azure-identity


In [30]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model

ml_client = MLClient(DefaultAzureCredential(), "e835122e-790c-40f9-99c6-98bfd4cdf02d", "Portfolio", "AzurePort")

model = Model(path="../model/churn_model.pkl", name="churn-model", description="Customer churn prediction model")
ml_client.models.create_or_update(model)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'churn-model', 'description': 'Customer churn prediction model', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/e835122e-790c-40f9-99c6-98bfd4cdf02d/resourceGroups/Portfolio/providers/Microsoft.MachineLearningServices/workspaces/AzurePort/models/churn-model/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/east2/code/Users/support/Customer_churn/src', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x78fffc9dbd90>, 'serialize': <msrest.serialization.Serializer object at 0x78fffc9da8c0>, 'version': '4', 'latest_version': None, 'path': 'azureml://subscriptions/e835122e-790c-40f9-99c6-98bfd4cdf02d/resourceGroups/Portfolio/workspaces/AzurePort/datastores/workspaceblobstore/paths/LocalUpload/c805fdef3c1031a0306

In [27]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration
)
import datetime

endpoint_name = f"churn-endpoint-{datetime.datetime.now().strftime('%m%d%H%M')}"

# Create endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Predicts customer churn probability",
    auth_mode="key"
)
ml_client.begin_create_or_update(endpoint).result()

# Create environment
env = Environment(
    name="churn-env",
    conda_file="../environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"
)



In [33]:
# Deployment
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(code=".", scoring_script="score.py"),
    instance_type="Standard_DS1_v2",
    instance_count=1
)

ml_client.begin_create_or_update(deployment).result()

# Set traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()



Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint churn-endpoint-10122230 exists
Uploading src (0.06 MBs): 100%|██████████| 56266/56266 [00:00<00:00, 2003163.59it/s]




...........................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://churn-endpoint-10122230.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://churn-endpoint-10122230.eastus2.inference.ml.azure.com/swagger.json', 'name': 'churn-endpoint-10122230', 'description': 'Predicts customer churn probability', 'tags': {}, 'properties': {'createdBy': 'Victor Isuo', 'createdAt': '2025-10-12T22:30:29.623401+0000', 'lastModifiedAt': '2025-10-12T23:11:00.746937+0000', 'azureml.onlineendpointid': '/subscriptions/e835122e-790c-40f9-99c6-98bfd4cdf02d/resourcegroups/portfolio/providers/microsoft.machinelearningservices/workspaces/azureport/onlineendpoints/churn-endpoint-10122230', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/e835122e-790c-40f9-99c6-98bfd4cdf02d/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:4cb43ab5-aae4-49ff-b43e-023139cd92a0:2bb3b1f7-0b1d-4f36-8c98-a95a4da1dd4

In [34]:
import requests, json

endpoint = ml_client.online_endpoints.get(name=endpoint_name)
scoring_uri = endpoint.scoring_uri
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key

sample_input = {"data": [[0, 0, 1, 0, 12, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 1, 2, 75.5, 1200.25]]}
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {primary_key}"}

response = requests.post(scoring_uri, headers=headers, data=json.dumps(sample_input))
print(response.status_code)
print(response.json())

200
{'error': 'The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()'}


In [22]:
print(X.columns.to_list())
print(len(X.columns))

['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
19
